# LIFE Star Catalog
## Introduction
One of the main use cases of the LIFE Target Database is to enable the creation of the LIFE Star Catalog (LIFE-StarCat). It is the stellar sample of potential targets that is used by LIFEsim to provide mission yield estimates. In this tutorial we guide you through how the 4th version of this catalog was created.

## Getting started
We start by importing the required Python modules for this tutorial.

In [1]:
import numpy as np # Used for arrays
import pyvo as vo # Used for catalog query
import astropy as ap # Used for votables
import math
import sys

Next we import the life_td_data_generation specific functions from the modules.

In [3]:
# Self created modules
import utils as hf
from provider import query

## Querying the Database
We will now query the LIFE Target Database (life_td) for the data needed to create the catalog. We specify the distance cut as 30 parsec according to the latest LIFE catalog version (4).

In [4]:
distance_cut=30.

Now we define the concrete query. It is one of the provided examples at http://dc.zah.uni-heidelberg.de/life/q/ex/examples. And also give the link for the service.

In [5]:
adql_query="""
    SELECT o.main_id, sb.coo_ra, sb.coo_dec, sb.plx_value, sb.dist_st_value,
        sb.sptype_string, sb.coo_gal_l, sb.coo_gal_b, sb.teff_st_value, 
        sb.mass_st_value, sb.radius_st_value, sb.binary_flag, sb.mag_i_value, 
        sb.mag_j_value,  sb.class_lum, sb.class_temp, 
        o_parent.main_id AS parent_main_id, sb_parent.sep_ang_value
    FROM life_td.star_basic AS sb
    JOIN life_td.object AS o ON sb.object_idref=o.object_id
    LEFT JOIN life_td.h_link AS h ON o.object_id=h.child_object_idref
    LEFT JOIN life_td.object AS o_parent ON 
        h.parent_object_idref=o_parent.object_id
    LEFT JOIN life_td.star_basic AS sb_parent ON 
        o_parent.object_id=sb_parent.object_idref
    WHERE o.type = 'st' AND sb.dist_st_value < """+str(distance_cut) 
service='http://dc.zah.uni-heidelberg.de/tap'

The above query returns parameters for all objects of type star with distances smaller than 30 pc.

Next we perform the actual query.

In [12]:
catalog=query(service,adql_query)

Let's have a short look at what we got:

In [13]:
print(catalog)

    main_id           coo_ra             coo_dec       plx_value ... class_temp parent_main_id sep_ang_value
                       deg                 deg            mas    ...                               arcsec   
--------------- ------------------ ------------------- --------- ... ---------- -------------- -------------
    *  61 Cyg B  316.7302660185276   38.74204403329556  286.0054 ...          K      *  61 Cyg         839.8
    *  61 Cyg A  316.7247482895925   38.74941731943694  285.9949 ...          K      *  61 Cyg         839.8
    L 1578-44 B  324.1601460120721  39.455739652570564   47.9104 ...                 L 1578-44           1.0
    L 1578-44 A 324.16073469423293  39.455702049799726   48.1082 ...          M      L 1578-44           1.0
UPM J2325+4717B  351.4090302929913   47.28469225673499   40.4566 ...            UPM J2325+4717            --
UPM J2325+4717A  351.4091812460812  47.284885878289714   40.1923 ...            UPM J2325+4717            --
     Ross  200B  32

We can see that we obtained a table of about 10'000 stars.

## Processing the result
### Removing non main-sequence stars
For the LIFE mission we are only interested in main sequence stars so in the next step we remove all other stars. We first remove those without main sequence temperature classes (e.g. white dwarfs). 

In [35]:
ms_tempclass=np.array(['O','B','A','F','G','K','M'])
cat=catalog[np.where(np.in1d(catalog['class_temp'],ms_tempclass))]

In [38]:
print(cat['main_id', 'mass_st_value','sptype_string'][np.where(cat['class_lum']=='')])

        main_id           mass_st_value    sptype_string
                             solMass                    
----------------------- ------------------ -------------
              G 227-48B               0.27         M3.51
               G 221-27              0.184            M4
              G  96-29B              0.184           M4:
            * mu. Dra C               0.27           M3:
             HD 154712B               0.73            K4
          LP   97-674 A                0.4          M2.2
       RX J1206.9+7007A               0.27          M3.5
              * i Boo B                0.9           G9:
             Ross  110B               0.27          M3.5
             Ross  110A               0.27          M3.5
             HD  13513A               0.54          M0.0
             G 274-138B              0.184            M4
                    ...                ...           ...
               G 129-49               0.27         M3.44
    ASAS J091728+1348.6        

Now for the final catalog we want to have stellar mass values for each object. Currently the database is not able to assign model masses to some of the spectral types. This is the case when instad of M3.0V something like dM3.0, M or M (3) V is given in sptype_string. This will be implemented in one of the future database releases but for now we will just remove all objects without entries in mass_st_value.

In [25]:
cat.remove_rows(cat['mass_st_value'].mask.nonzero()[0])

Next we remove objects that are not main sequence in luminocity class.

In [ ]:
ms_lumclass=np.array(['V'])
temp=catalog[np.where(np.in1d(catalog['class_lum'],ms_lumclass))]
print(temp)
print(temp[temp['mass_st_value'].mask.nonzero()[0]])

wait, we want to assume main sequence if nothing is given... and then I could also get masses for them... but I wouldn't necessary have it in the db...
I faintly remember we were also interested in IV stars, but can't remember why. tried to look it up in the paper but couldn't find it. maybe in the report?
"We did, however, keep the ones with no luminocity class given assuming them to be main sequence stars. This is justified as the main sequence is the longest lasting evolutionary period of a star leaving the great majority of stars in this stage." states why the assumption of main sequence is ok.
found out we only included main sequence for modeling reasons. so I need to exclude all else and therefore also found a bug from StarCat4

The stellar sample was compiled from querying the LIFE database (Menti et al. 2021) for stellar objects within 30 pc. Appart from the new source we tried to keep the criteria for inclusion into the catalog as similar to the ones from the previous LIFE catalog version as possible. This allows us to compare the two versions better before changing the criteria in the next catalog version (LIFE-StarCat5) again. (Among others to include higher order systems since they could have better chances for hosting life but in current version not included due to complexity e.g. centaury system). We started by removed objects with luminosity class I, II and III as we focused on main-sequence stars (or close to). In case no luminosity class was given we assumed the objects were main-sequence objects. In order to assess which objects are members of binary or multiple systems we used the LIFE database's hierarchical link. This feature connects an object with its parent and child objects. We therefore removed objects which were neither single stars nor trivial binaries. Concretely we removed objects where either the parent object of the star is istself a child object of another system, where multiple parent objects were given for the same star or which had more than one companion. As a note of caution this classification in the LIFE database is heavily based on Washington Visual Double Star Catalog (WDS; Mason et al. 2001) which in term only describes visual multiples and therefore contains some objects that are not physically bound. We also removed objects with incomplete information for the stellar parameters (this step also included binaries, where one component was not listed as a main-sequence star) or no separation between the binaries was given. In order to ensure that all stellar components of the remaining binary systems could harbor stable planetary systems between 0.5 and 10 AU we transformed the angular separation into physical and used it together with the stability criterion from Holman & Wiegert (1999) that takes into account the stellar masses, their separation and the eccentricity of the binary orbit (which we assumed to be zero). Systems that did not fulfill this stability criterion were removed. Finally we introduced a new parameter for future mission architecture analysis in which we flag objects that are contained within \pm 45 degrees of the ecliptic plane. This simulates the loss of targets by not full sky field of view. The catalog consists of 3909 stars in total (434 wide binary components and 3475 single stars).

In [ ]:
def crit_sep(eps,mu,a_bin):
    """
    Computes critical semimajor-axis for planet orbit stability.

    For binary system as described in Holman and Wiegert 1999.

    :param eps: Binary orbit excentricity.
    :type eps:
    :param mu: mass fraction with mu=m_s/(m_p+m_s), with m_s the mass 
        of the star considered as perturbing binary companion and m_p 
        the mass of the star the planet is orbiting.
    :type mu:
    :param a_bin: semimajor-axis of the binary stars.
    :type a_bin:
    :returns: Critical separation beyond which a planet on a S-type 
        orbit (circumstellar) and on a P-type orbit (circumbinary) is 
        not stable any more.
    :rtype: 
    """
    a_crit_s=(0.464-0.38*mu-0.631*eps+0.586*mu*eps+0.15*eps**2\
              -0.198*mu*eps**2)*a_bin
    a_crit_p=(1.6+5.1*eps-2.22*eps**2+4.12*mu-4.27*eps*mu-5.09*mu**2\
              +4.61*eps**2*mu**2)*a_bin
    return a_crit_s,a_crit_p

def ecliptic(ang,ra,dec):
    """
    Computes if position is within angle from the ecliptic.

    :param ang: Angle in degrees.
    :type ang:
    :param ra: Right ascention in degrees.
    :type ra: np.array
    :param dec: Array of declination in degrees.
    :type dec: np.array
    :returns: Flags.
    :rtype: np.array
    """
    ecliptic=(23.4)*np.sin(2*np.pi*ra/360)
    flag=['True' if dec[j] > -ang+ecliptic[j] and dec[j] < ang+ecliptic[j] \
            else 'False' for j in range(len(ra))]
    return flag

def StarCat_creation(plx_cut,stars=[],details=False,querying=True):
    """
    LIFE-StarCat4 creation
    
    :param plx_cut:
    :type plx_cut:
    :param
    :type
    :param
    :type
    :param
    :type
    :returns:
    :rtype:
    """
    
    # version 3 parameters were: ra, dec, plx, distance, name, sptype, 
    # coo_gal_l, coo_gal_b, Teff, R, M, sep_phys, binary_flag, mag_i, 
    # mag_j
    adql_query="""
    SELECT o.main_id, sb.coo_ra, sb.coo_dec, sb.plx_value, sb.dist_st_value,
        sb.sptype_string, sb.coo_gal_l, sb.coo_gal_b, sb.teff_st_value, 
        sb.mass_st_value, sb.radius_st_value, sb.binary_flag, sb.mag_i_value, 
        sb.mag_j_value,  sb.class_lum, sb.class_temp, 
        o_parent.main_id AS parent_main_id, sb_parent.sep_ang_value
    FROM life_td.star_basic AS sb
    JOIN life_td.object AS o ON sb.object_idref=o.object_id
    LEFT JOIN life_td.h_link AS h ON o.object_id=h.child_object_idref
    LEFT JOIN life_td.object AS o_parent ON 
        h.parent_object_idref=o_parent.object_id
    LEFT JOIN life_td.star_basic AS sb_parent ON 
        o_parent.object_id=sb_parent.object_idref
    WHERE o.type = 'st' AND sb.dist_st_value < """+str(plx_cut) 
    # we are only interested in object type stars, up to a distance cut 
    # and well defined luminocity class (to sort out objects not around 
    # main sequence)
    service='http://dc.zah.uni-heidelberg.de/tap'
    print(f'Getting stars with distance smaller than {plx_cut}...')
    
    if querying==True:
        print('Querying life_td...')
        cat=query(service,adql_query)
        hf.save([cat],['StarCat4_raw'],location='data/')
    else:
        print('Loading StarCat4_raw...')
        [cat]=hf.load(['StarCat4_raw'],location='data/')
    
    if stars!=[]:
        stars=np.array(stars)
        stars=hf.object_contained(stars,cat['main_id'],details)
        
    print('Removing objects whose temperature class is not in OBAFGKM...')
    cat=cat[np.where(cat['class_temp']!='')]
    cat.remove_rows(cat['mass_st_value'].mask.nonzero()[0])
    
    if len(stars)>0:
        stars=hf.object_contained(stars,cat['main_id'],details)
    
    print('Removing objects where luminocity I, II or III...')
    cat=cat[np.where(cat['class_lum']!='I')]
    cat=cat[np.where(cat['class_lum']!='II')]
    cat=cat[np.where(cat['class_lum']!='III')]
    
    #separating sample
    singles=cat[np.where(cat['binary_flag']=='False')]
    multiples=cat[np.where(cat['binary_flag']=='True')]

    print('Removing higher order multiples:')
    print('... where parent object is a child object as well.')
    #query h_link
    adql_query2="""
    SELECT o.main_id as child_main_id,o.object_id
    FROM life_td.object AS o
    JOIN life_td.h_link AS h on o.object_id=h.child_object_idref
    """
    if querying==True:
        print('Querying life_td for h_link...')
        h_link=query(service,adql_query2)
        hf.save([cat],['h_link'],location='data/')
    else:
        print('Loading h_link...')
        [h_link]=hf.load(['best_h_link'],location='data/')
    #remove those where parent_main_id in h_link as child_main_id
    higher_order_multiples=np.in1d(multiples['parent_main_id'],\
            h_link['child_main_id'])
    # all without higher order multiples
    multiples=multiples[np.invert(higher_order_multiples)]
    
    if len(stars)>0:
        stars=hf.object_contained(stars,
                ap.table.vstack([singles,multiples])['main_id'],details)
    
    print('... where multiple parents or more than two components given.')
    double=[]
    grouped=multiples.group_by('main_id')
    ind=grouped.groups.indices
    for i in range(len(ind)-1):
        if ind[i+1]-ind[i]!=1:
        #print(ind[i],ind[i+1])
            double.append(grouped['main_id'][ind[i]])
    multiples=grouped[np.where(np.invert(np.in1d(grouped['main_id'],double)))]

    if len(stars)>0:
        stars=hf.object_contained(stars,
                ap.table.vstack([singles,multiples])['main_id'],details)
        
    #critical sep:
    print('Removing those binaries where no separation value given...')
    multiples=multiples[np.where(multiples['sep_ang_value'].mask==False)].copy()
    
    if len(stars)>0:
        stars=hf.object_contained(stars,
                ap.table.vstack([singles,multiples])['main_id'],details)
    
    print('Transforming from angular separation into physical one...') 
    #transform into physical separations the remaining ones

    multiples['sep_phys_value']=multiples['sep_ang_value']
    multiples['sep_phys_value'].unit=ap.units.AU
    for i in range(len(multiples)):
        multiples['sep_phys_value'][i]=np.round(
                multiples['sep_ang_value'][i]*multiples['dist_st_value'][i],1)
    
    
    print('Keeping only binaries where both components fulfill requirements.') 
    # meaning having the parameters we need for critical sep 
    # computation (main sequence & sep given)

    grouped_multiples=multiples.group_by('parent_main_id')
    ind=grouped_multiples.groups.indices

    result=grouped_multiples[:0].copy()

    for i in range(len(ind)-1):
        l=ind[i+1]-ind[i]
        if l==2:
            result.add_row(grouped_multiples[ind[i]])
            result.add_row(grouped_multiples[ind[i]+1])
    
    if len(stars)>0:
        stars=hf.object_contained(stars,
                ap.table.vstack([singles,result])['main_id'],details)
    
    print('Keeping only objects where <10 AU planet orbits are stable...')
    
    result['a_crit_s']=result['sep_phys_value']

    for i in range(len(result)):
        m_p=result['mass_st_value'][i]
        if i % 2 == 0:
            m_s=result['mass_st_value'][i+1]
        else:
            m_s=result['mass_st_value'][i-1]
        mu=m_s/(m_p+m_s)
        result['a_crit_s'][i]=crit_sep(0,mu,result['sep_phys_value'][i])[0]
        #assumed circular orbit and sep_phys = a_bin

    final=result[:0].copy()
    #wait, didn't I already define this? -> was before removing some
    ind=result.group_by('parent_main_id').groups.indices
    a_max=10.
    
    for i in range(len(ind)-1):
        if a_max < min(result['a_crit_s'][ind[i]],result['a_crit_s'][ind[i]+1]):
            final.add_row(result[ind[i]])
            final.add_row(result[ind[i]+1])
    
    if len(stars)>0:
        stars=hf.object_contained(stars,
                ap.table.vstack([singles,final])['main_id'],details)
        
    #does not look right. a_crit is 8 but stayed in sample??

    #what about the inner limit of 0.5??
    
    StarCat4=ap.table.vstack([singles,final])
    
    print('Adding architecture parameter...')
    # flag any object whose declination is contained within the region 
    # between -(23.4+45)*sin(RA) and +(23.4+45)*sin(RA) with the 
    # object's RA in degrees.

    StarCat4['ecliptic_pm45deg']=ecliptic(45,StarCat4['coo_ra'],\
            StarCat4['coo_dec'])
    hf.save([StarCat4],['StarCat4'],location='data/')
    
    if len(stars)>0:
        print('All criteria were met by:',len(stars))
        if details:
            print(stars)
    
    return StarCat4
    